<a href="https://colab.research.google.com/github/oscarserc/LENGUAJE_NLP/blob/main/Transformers_5_HuggingFace_Generacion_de_RESUMENES_de_TEXTO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# -----------------------------------------------------------
# PRACTICAS DE: Natural Language Processing with Transformers  - CAPITULO 6
# -----------------------------------------------------------

# GitHub del libro: https://github.com/nlp-with-transformers/notebooks

# Uncomment and run this cell if you're on Colab or Kaggle
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
from install import *
install_requirements()



Cloning into 'notebooks'...
remote: Enumerating objects: 422, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 422 (delta 0), reused 5 (delta 0), pack-reused 416
Receiving objects: 100% (422/422), 24.97 MiB | 11.94 MiB/s, done.
Resolving deltas: 100% (190/190), done.
/content/notebooks
⏳ Installing base requirements ...
✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!


In [ ]:

from utils import *
setup_chapter()


Using transformers v4.11.3
Using datasets v1.16.1


In [ ]:

from transformers import pipeline, set_seed


#**Realización de RESÚMENES**.
#Base de Datos CNN/DailyMail

In [ ]:

from datasets import load_dataset

# ------------------------------------
#   CORRECCIÓN del ERROR de descarga
# ------------------------------------

# Por tema de derechos o algo similar, no se puede descargar como se indica.
# HUGGING FACE:                https://huggingface.co/datasets/cnn_dailymail
#  --> Tiene una página para tener acceso a estos datos: https://huggingface.co/datasets/ccdv/cnn_dailymail
#      Clickando en el lateral de la página de información:
#                                 <\> use in dataset library
#      aparece una ventana flotante con el código que podemos emplear incrustándolo en nuestro programa
#                              load_dataset("ccdv/cnn_dailymail")

# dataset = load_dataset('cnn_dailymail', version='3.0.0')
dataset = load_dataset('ccdv/cnn_dailymail', version='3.0.0')
print(f"Features: {dataset['train'].column_names}")


  0%|          | 0/5 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/159M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376M [00:00<?, ?B/s]

  0%|          | 0/5 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/ccdv___cnn_dailymail/default/3.0.0/0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Features: ['article', 'highlights', 'id']


In [ ]:

sample = dataset['train'][1]
print(f""" Article (excerpt of 500 characters, total length: {len(sample['article'])}): """)
print(sample['article'][:500])
print(f"\nSummary (length: {len(sample['highlights'])}): ")
print(sample['highlights'])


 Article (excerpt of 500 characters, total length: 3192):
(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his
third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m
relay. The fastest man in the world charged clear of United States rival Justin
Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel
Ashmeade and Bolt won in 37.36 seconds. The U.S finished second in 37.56 seconds
with Canada taking the bronze after Britain were disqualified for a faulty
handover. The 26-year-old Bolt has n

Summary (length: 180):
Usain Bolt wins third gold of world championship .
Anchors Jamaica to 4x100m relay victory .
Eighth gold at the championships for Bolt .
Jamaica double up in women's 4x100m relay .


#**Pipelines para la realización de RESÚMENES de texto**.

In [ ]:

sample_text = dataset['train'][1]['article'][:2000]
# Recolectamos todos los RESÚMENES generados con cada modelo en un DICCIONARIO
summaries = {}


In [ ]:

import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:

string = 'The U.S. are a country. The U.N. is an organization.'
sent_tokenize(string)


['The U.S. are a country.', 'The U.N. is an organization.']

#Resumen Baseline.

In [ ]:

def three_sentence_summary(text):
  return "\n".join(sent_tokenize(text)[:3])


In [ ]:

summaries['baseline'] = three_sentence_summary(sample_text)
summaries['baseline']


"(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay.\nThe fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds.\nThe U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover."

#Seguimos con el código de algunos MODELOS con los que probar.
#(*Los más grandes como GPT-2 pueden agotar la memoria, por lo que se quedan cortos los recursos asignados por COOLAB*). 

#**GPT-2**.

In [ ]:

# Natural Language Processing with Transformers (pag. 143)
# --------------------------------------------------------
#
# Si se queda corto de memoria, se puede probar a cargar un modelo más ligero.
#
# Probar por ejemplo, checkpoints como: 'gpt' o 't5-small'

from transformers import pipeline, set_seed

set_seed(42)
pipe = pipeline('text-generation', model='gpt2-xl')
gpt2_query = sample_text + '\nTL;DR:\n'
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
summaries['gpt2'] = '\n'.join(sent_tokenize(pipe_out[0]['generated_text'][len(gpt2_query) :]))


#**T5**.

In [ ]:

pipe = pipeline('summarization', model='t5-large')
pipe_out = pipe(sample_text)
summaries['t5'] = '\n'.join(sent_tokenize(pipe_out[0]['summary_text']))


#**BART**.

In [ ]:

pipe = pipeline('summarization', model='facebook/bart-large-cnn')
pipe_out = pipe(sample_text)
summaries['bart'] = '\n'.join(sent_tokenize(pipe_out[0]['summary_text']))


#**PEGASUS**.

In [ ]:

pipe = pipeline('summarization', model='google/pegasus-cnn_dailymail')
pipe_out = pipe(sample_text)
summaries['pegasus'] = pipe_out[0]['summary_text'].replace(' .<n>', '.\n')


#Comparación.

In [ ]:

print('GROUND TRUTH')
print(dataset['train'][1]['highlights'])
print('')

for model_name in summaries:
  print(model_name.upper())
  print(summaries[model_name])
  print('')


#**Medición de la calidad del texto generado**.
#--> **BLEU** (*Bilingual Evaluation Understudy*).

In [ ]:

from datasets import load_metric

bleu_metric = load_metric('sacrebleu')


Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

In [ ]:

import pandas as pd
import numpy as np

bleu_metric.add(
    prediction='the the the the the the', reference=['the cat is on the mat'])
results = bleu_metric.compute(smooth_method='floor', smooth_value=0)
results['precisions'] = [np.round(p, 2) for p in results['precisions']]
pd.DataFrame.from_dict(results, orient='index', columns=['value'])


,value
score,0.0
counts,"[2, 0, 0, 0]"
totals,"[6, 5, 4, 3]"
precisions,"[33.33, 0.0, 0.0, 0.0]"
bp,1.0
sys_len,6
ref_len,6


In [ ]:

bleu_metric.add(
    prediction='the cat is on mat', reference=['the cat is on the mat']
)
results = bleu_metric.compute(smooth_method='floor', smooth_value=0)
results['precisions'] = [np.round(p, 2) for p in results['precisions']]
pd.DataFrame.from_dict(results, orient='index', columns=['Value'])


,Value
score,57.893007
counts,"[5, 3, 2, 1]"
totals,"[5, 4, 3, 2]"
precisions,"[100.0, 75.0, 66.67, 50.0]"
bp,0.818731
sys_len,5
ref_len,6


#--> **ROUGE** (*Recall-Oriented Understudy for Gisting Evaluation*).

In [ ]:

# Cargamos la MÉTRICA
rouge_metric = load_metric('rouge')


Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

In [ ]:

reference   = dataset['train'][1]['highlights']
records     = []
rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']

# EJECUTADO SÓLO LA BASELINE por si da error al agotarse recursos
# (sólo aparecerá la linea: baseline)
for model_name in summaries:
  rouge_metric.add(prediction=summaries[model_name], reference=reference)
  score = rouge_metric.compute()
  rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
  records.append(rouge_dict)
pd.DataFrame.from_records(records, index=summaries.keys())  


,rouge1,rouge2,rougeL,rougeLsum
baseline,0.303571,0.090909,0.214286,0.232143


#**Evaluación de PEGASUS con el dataset CNN/DailyMail**.

In [ ]:

# PÁGINA EN HUGGING FACE: https://huggingface.co/docs/transformers/model_doc/pegasus
# ----------------------------------------------------------------------------------

import matplotlib.pyplot as plt
import pandas as pd
from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# dataset = load_dataset('ccdv/cnn_dailymail', version='3.0.0')
rouge_metric = load_metric('rouge', cache_dir=None)
rouge_names  = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']


In [ ]:

def evaluate_summaries_baseline(dataset, metric,
                                column_text = 'article',
                                column_summary = 'highlights'):
  summaries = [three_sentence_summary(text) for text in dataset[column_text]]
  metric.add_batch(predictions=summaries,
                   references=dataset[column_summary])
  score     = metric.compute()
  return score


In [ ]:

test_sampled = dataset['test'].shuffle(seed=42).select(range(1000))

score = evaluate_summaries_baseline(test_sampled, rouge_metric)
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame.from_dict(rouge_dict, orient='index', columns=['baseline']).T


,rouge1,rouge2,rougeL,rougeLsum
baseline,0.387835,0.170826,0.247057,0.35445


#Código de los TRANSFORMERS.

In [ ]:

from tqdm import tqdm
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

def chunks(list_of_elements, batch_size):
  # Se hace un YIELD con fragmentos del tamaño 'batch_size' sucesivamente de 'list_of_elements'
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i : i + batch_size]

def evaluate_summaries_pegasus(dataset, metric, model, tokenizer,
                               batch_size=16, device=DEVICE,
                               column_text    = 'article',
                               column_summary = 'highlights'):
  articles_batches = list(chunks(dataset[column_text], batch_size))
  target_batches   = list(chunks(dataset[column_summary], batch_size))

  for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
    inputs            = tokenizer(article_batch, max_length=1024, truncation=True, padding='max_length', return_tensors='pt')
    summaries         = model.generate(input_ids=inputs['input_ids'].to(DEVICE), attention_mask=inputs['attention_mask'].to(DEVICE),
                               length_penalty=0.8, num_beams=8, max_length=128)
    decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]
    decoded_summaries = [d.replace('<n>', ' ') for d in decoded_summaries]
    metric.add_batch(predictions=decoded_summaries, references=target_batch)

  score = metric.compute()
  return score     


In [ ]:

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = 'google/pegasus-cnn_dailymail'
tokenizer  = AutoTokenizer.from_pretrained(model_ckpt)
 
model      = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(DEVICE)
score      = evaluate_summaries_pegasus(test_sampled, rouge_metric, model, tokenizer, batch_size=8)

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=['pegasus'])


In [ ]:

pd.DataFrame(rouge_dict, index=['pegasus'])


#**Entrenamiento del MODELO capaz de realizar RESÚMENES de textos**.

In [ ]:

# BASE DE DATOS: SAMsum
# ---------------------
#
# Creada por SAMSUNG, contiene textos y resúmenes.
# Se emplea en tareas de generación de RESÚMENES (summary)

dataset_samsum = load_dataset('samsum')
split_lengths  = [len(dataset_samsum[split]) for split in dataset_samsum]

print(f'Split lengths: {split_lengthes}')
print(f'Features: {dataset_samsum['train'].column_names}')

print('\nDialogue: ')
print(dataset_samsum['test'][0]['dialogue'])
print('\nSummary: ')
print(dataset_samsum['test'][0]['summary'])


In [ ]:

print(f'Split lengths: {split_lengths}')
print(f'Features: {dataset_samsum['train'].column_names}')

print('\nDialogue: ')
print(dataset_samsum['test'][0]['dialogue'])
print('\nSummary: ')
print(dataset_samsum['test'][0]['summary'])


#**Evaluando PEGASUS con la base de datos SAMsum**.

In [ ]:

pipe_out = pipe(dataset_samsum['test'][0]['dialogue'])
print('Summary: ')
print(pipe_out[0]['summary_text'].replace(' .<n>', '\n'))


In [ ]:

score = evaluate_summaries_pegasus(dataset_samsum['test'], rouge_metric, model,
                                   tokenizer, column_text='dialogue',
                                   column_summary='summary', batch_size=8)

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=['pegasus'])


In [ ]:

pd.DataFrame(rouge_dict, index=['pegasus'])


#**Fine-tuning de PEGASUS**.

In [ ]:

d_len = [len(tokenizer.encode(s)) for s in dataset_samsum['train']['dialogue']]
s_len = [len(tokenizer.encode(s)) for s in dataset_samsum['train']['dailogue']]

fig, axes = plt.subplots(1, 2, figsize=(10, 3.5), sharey=True)

axes[0].hist(d_len, bins=20, color='C0', edgecolor='C0')
axes[0].set_title('Dialogue Token Length')
axes[0].set_xlabel('Length')
axes[0].set_ylabel('Cout')

axes[1].hist(s_len, bins=20, color='C0', edgecolor='C0')
axes[1].set_title('Summary Token Length')
axes[1].set_xlabel('Length')

plt.tight_layout()
plt.show()


In [ ]:

def convert_examples_to_features(example_batch):
  input_encodings = tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example_batch['summary'], max_length=128, truncation=True)

  return {'input_ids'     : input_encodings['input_ids'],
          'attention_mask': input_encodings['attention_mask'],
          'labels'        : target_encodings['input_ids']}

dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched=True)
columns           = ['input_ids', 'labels', 'attention_mask']
dataset_samsum_pt.set_format(type='torch', columns=columns)            


In [ ]:

text = ['PAD', 'Transformers', 'are', 'awesome', 'for', 'text', 'summarization']
rows = []

for i in range(len(text)-1):
  rows.append({'step': i+1, 'decoder_input': text[:i+1], 'label': text[i+1]})

pd.DataFrame(rows).set_index('step')
